In [1]:
import os
import torch
from torch._logging import set_logs
import logging

In [1]:

from opensora.models.stdit import STDiT3
import pickle
from opensora.sg_profiler import sg_profiler
import wat
import torch
import torch._dynamo as dynamo
import time

model: STDiT3 = torch.load('../torchlog/debug_pickle/STDiT3_model.pt')
z_in, t, model_args = pickle.load(open('../torchlog/debug_pickle/fwd_params.pkl', 'rb'))
with torch.no_grad():
    # warmup
	model(z_in, t, **model_args);
	# initialize cudagraphs, triton, etc.
	model(z_in, t, **model_args);
 
	time_start = time.time()
	for _ in range(10):
		model(z_in, t, **model_args);
	time_end = time.time()
print(f'Time taken: {((time_end - time_start) / 10) * 1000:.2f} ms / iter')

/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/colossalai/pipeline/schedule/_utils.py:19: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _register_pytree_node(OrderedDict, _odict_flatten, _odict_unflatten)
/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/torch/utils/_pytree.py:332: UserWarning: <class 'collections.OrderedDict'> is already registered as pytree node. Overwriting the previous registration.
  warnings.warn(
/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/colossalai/legacy/registry/__init__.py:1: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  import torch.distributed.optim as dist_optim
/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/colossalai/accelerator/cuda_accele

Time taken: 153.14 ms / iter


In [2]:
spatial_block = model.spatial_blocks[0]
drop_path = spatial_block.drop_path
# random tensor
random_tensor = torch.randn(10, 14, 16, 25)
# apply drop path
dropped_tensor = drop_path(random_tensor)
print(torch.allclose(dropped_tensor, random_tensor))

True


In [2]:
time_start = time.time()
n = 10
with torch.no_grad():
	for _ in range(n):
		model(z_in, t, **model_args);
time_end = time.time()
print(f'Time taken: {((time_end - time_start) / n) * 1000:.2f} ms / iter')

Time taken: 157.50 ms / iter


In [11]:
for x in model.spatial_blocks: print('spatial block.drop_path', x.drop_path)
for x in model.temporal_blocks: print('temporal block.drop_path', x.drop_path)

spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
spatial block.drop_path Identity()
temporal block.drop_

In [4]:
print("\n".join(torch._inductor.list_options()))

TYPE_CHECKING
debug
disable_progress
verbose_progress
fx_graph_cache
fx_graph_remote_cache
autotune_local_cache
autotune_remote_cache
force_disable_caches
cpp_wrapper
abi_compatible
c_shim_version
dce
static_weight_shapes
size_asserts
nan_asserts
pick_loop_orders
inplace_buffers
allow_buffer_reuse
memory_planning
memory_pool
benchmark_harness
epilogue_fusion
epilogue_fusion_first
pattern_matcher
post_grad_custom_pre_pass
post_grad_custom_post_pass
joint_custom_pre_pass
joint_custom_post_pass
pre_grad_custom_pass
split_cat_fx_passes
efficient_conv_bn_eval_fx_passes
is_predispatch
group_fusion
batch_fusion
pre_grad_fusion_options
post_grad_fusion_options
reorder_for_locality
dynamic_scale_rblock
force_fuse_int_mm_with_mul
use_mixed_mm
fx_passes_numeric_check
force_mixed_mm
reorder_for_compute_comm_overlap
reorder_for_compute_comm_overlap_passes
estimate_op_runtime
intra_node_bw
inter_node_bw
max_autotune
max_autotune_pointwise
max_autotune_gemm
force_same_precision
max_autotune_gemm_back

In [2]:
def run_flops_profiler():
    # Don't import at the start bc it will mess with the compiler
	from flops_profiler.profiler import FlopsProfiler
	flops_profiler = FlopsProfiler(model)
	
	from flops_profiler.profiler import cuda_sync
	assert cuda_sync, 'cuda_sync should be True'
	flops_profiler.start_profile()
	model(z_in, t, **model_args)
	flops_profiler.stop_profile()
	print('Duration:', flops_profiler.get_total_duration(True))
	print('FLOPS:', flops_profiler.get_total_flops(True))
	print('MACS:', flops_profiler.get_total_macs(True))
	print('Params:', flops_profiler.get_total_params(True))
	return flops_profiler
fp = run_flops_profiler()
fp.print_model_profile()

/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/torch/_dynamo/variables/functions.py:663: UserWarning: Graph break due to unsupported builtin time.time. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.
  torch._dynamo.utils.warn_once(msg)


Unsupported: Graph break due to unsupported builtin time.time. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.

from user code:
   File "/scr/govande/opensora/opensora/models/stdit/stdit3.py", line 368, in compiled_blocks
    x = spatial_block(x, y, t_mlp, y_lens, x_mask, t0_mlp, T, S)
  File "/scr/govande/miniconda3/envs/opensora2/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1592, in _call_impl
    args_result = hook(self, args)
  File "/scr/govande/miniconda3/envs/opensora2/lib/python3.1

In [2]:
print('model.enable_flash_attn:', model.enable_flash_attn)
print('model.enable_layernorm_kernel:', model.enable_layernorm_kernel)

model.enable_flash_attn: True
model.enable_layernorm_kernel: False


In [2]:
def run_torch_profiler():
	with torch.no_grad():
		with sg_profiler('stdit3_cudagraphs') as prof:
			for _ in range(5): # More steps than needed
				model(z_in, t, **model_args)
				prof.step()
	return prof
prof = run_torch_profiler()

W0930 21:47:36.646768 139700641792384 torch/fx/experimental/symbolic_shapes.py:4449] [1/1] r0 is not in var_ranges, defaulting to unknown range.


KeyboardInterrupt: 

In [9]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by='self_device_time_total', row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls                                                                      Input Shapes   Total FLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------

In [ ]:
run_torch_profiler()

FLOPS: 25301.41 G
MACS: 12649.07 GMACs
